In [45]:
!pip3 install pandas
!pip3 install matplotlib pyarrow fastparquet pandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import matplotlib.dates as mdates
from matplotlib.lines import Line2D
import random

# Seed für Reproduzierbarkeit
random.seed(42)
np.random.seed(42)

# Matplotlib-Styling für wissenschaftlichen Stil
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'DejaVu Serif', 'Palatino'],
    'font.size': 12,
    'axes.labelsize': 14,
    'axes.titlesize': 16,
    'xtick.labelsize': 13,
    'ytick.labelsize': 13,
    'legend.fontsize': 13,
    'figure.dpi': 300,
    'savefig.dpi': 300,
    'axes.grid': True,
    'grid.alpha': 0.3,
    'lines.linewidth': 1.5,
})

def generate_synthetic_data(days=365):
    """Synthetische Preisdaten für zwei kointegrierte Assets generieren"""
    # Daten erstellen
    base_date = datetime(2023, 1, 1)
    dates = [base_date + timedelta(days=i) for i in range(days)]
    
    # Gemeinsamer Trend
    common_trend = np.cumsum(np.random.normal(0.0005, 0.005, days))
    
    # Individuelle Komponenten mit Mean-Reversion Spread
    noise1 = np.random.normal(0, 0.01, days)
    noise2 = np.random.normal(0, 0.01, days)
    
    # Mean-Reversion Spread erstellen
    spread = np.zeros(days)
    spread[0] = 0
    mean_reversion_strength = 0.05
    spread_volatility = 0.02
    
    for i in range(1, days):
        spread[i] = spread[i-1] - mean_reversion_strength * spread[i-1] + np.random.normal(0, spread_volatility)
    
    # Preise generieren
    price1 = 100 * np.exp(common_trend + noise1)
    price2 = 90 * np.exp(common_trend + noise2 + spread)
    
    # DataFrame erstellen
    df = pd.DataFrame({
        'date': dates,
        'price1': price1,
        'price2': price2,
    })
    df.set_index('date', inplace=True)
    
    return df

def calculate_zscore(df, window_size=20):
    """Z-Score des Preisverhältnisses berechnen"""
    # Preisverhältnis berechnen
    df['ratio'] = df['price1'] / df['price2']
    
    # Z-Score mit rollendem Mittelwert und Standardabweichung berechnen
    df['mean'] = df['ratio'].rolling(window=window_size).mean()
    df['std'] = df['ratio'].rolling(window=window_size).std()
    df['zscore'] = (df['ratio'] - df['mean']) / df['std']
    
    return df

def simulate_trades_exact(df, entry_threshold=2.0, exit_threshold=0.5, window_size=20):
    """Trades basierend auf Z-Score-Schwellenwerten simulieren"""
    trades = []
    in_position = False
    position_type = None
    
    # Crossover-Punkte für genaue Ein- und Ausstiege finden
    for i in range(window_size+1, len(df)):
        curr_zscore = df['zscore'].iloc[i]
        prev_zscore = df['zscore'].iloc[i-1]
        
        # Einstiegsbedingungen
        if not in_position:
            # Einstieg für Short A / Long B
            if prev_zscore <= entry_threshold and curr_zscore > entry_threshold:
                in_position = True
                position_type = 'short-long'
                trades.append({
                    'type': 'entry',
                    'position': position_type,
                    'date': df.index[i],
                    'zscore': entry_threshold
                })
            
            # Einstieg für Long A / Short B
            elif prev_zscore >= -entry_threshold and curr_zscore < -entry_threshold:
                in_position = True
                position_type = 'long-short'
                trades.append({
                    'type': 'entry',
                    'position': position_type,
                    'date': df.index[i],
                    'zscore': -entry_threshold
                })
        
        # Ausstiegsbedingungen
        elif in_position:
            # Ausstieg Short-Long Position
            if position_type == 'short-long' and prev_zscore >= exit_threshold and curr_zscore < exit_threshold:
                in_position = False
                trades.append({
                    'type': 'exit',
                    'position': position_type,
                    'date': df.index[i],
                    'zscore': exit_threshold
                })
            
            # Ausstieg Long-Short Position
            elif position_type == 'long-short' and prev_zscore <= -exit_threshold and curr_zscore > -exit_threshold:
                in_position = False
                trades.append({
                    'type': 'exit',
                    'position': position_type,
                    'date': df.index[i],
                    'zscore': -exit_threshold
                })
    
    return trades

# Hauptfunktion, die nur den dritten Plot erstellt
def create_zscore_plot():
    # Synthetische Daten generieren
    days = 365
    df = generate_synthetic_data(days)
    
    # Z-Score berechnen
    window_size = 20
    df = calculate_zscore(df, window_size)
    
    # Trades simulieren
    entry_threshold = 2.0
    exit_threshold = 0.5
    trades = simulate_trades_exact(df, entry_threshold, exit_threshold, window_size)
    
    # Z-Score Plot mit Ein-/Ausstiegspunkten
    plt.figure(figsize=(12, 5))
    plt.plot(df.index, df['zscore'], label='Z-Score', color='navy', linewidth=1.5)
    
    # Schwellenwertlinien hinzufügen
    plt.axhline(y=entry_threshold, color='crimson', linestyle='--', linewidth=1.2, label='Entry Threshold')
    plt.axhline(y=-entry_threshold, color='crimson', linestyle='--', linewidth=1.2)
    plt.axhline(y=exit_threshold, color='limegreen', linestyle='--', linewidth=1.2, label='Exit Threshold')
    plt.axhline(y=-exit_threshold, color='limegreen', linestyle='--', linewidth=1.2)
    plt.axhline(y=0, color='gray', linestyle='-', alpha=0.5)
    
    # Ein- und Ausstiegspunkte plotten
    for trade in trades:
        if trade['type'] == 'entry':
            if trade['position'] == 'short-long':
                marker = 'o'  # Kreis für Einträge
                color = 'magenta'
                size = 80
            else:  # long-short
                marker = 'o'  # Kreis für Einträge
                color = 'darkorange'
                size = 80
        else:  # exit
            marker = 'X'  # X für Ausstiege
            color = 'limegreen'
            size = 70
        
        # Marker am exakten Schwellenwert hinzufügen
        plt.scatter(trade['date'], trade['zscore'], marker=marker, color=color, s=size, zorder=5, 
                  edgecolors='black', linewidths=1)
    
    # Beschriftungen hinzufügen
    plt.ylabel('Z-Score')
    plt.grid(True, alpha=0.3)
    
    # X-Achse formatieren
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=2))  # Alle 2 Monate anzeigen
    
    # Benutzerdefinierte Legende für Trade-Marker erstellen
    handles, labels = plt.gca().get_legend_handles_labels()
    custom_lines = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor='magenta', markersize=8, 
              markeredgecolor='black', markeredgewidth=0.8, label='Short A / Long B'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='darkorange', markersize=8, 
              markeredgecolor='black', markeredgewidth=0.8, label='Long A / Short B'),
        Line2D([0], [0], marker='X', color='w', markerfacecolor='limegreen', markersize=8, 
              markeredgecolor='black', markeredgewidth=0.8, label='Exit')
    ]
    handles.extend(custom_lines)
    plt.legend(handles=handles, loc='upper right')
    
    plt.tight_layout()
    plt.savefig('figure3_zscore.pdf', format='pdf', bbox_inches='tight')
    plt.savefig('figure3_zscore.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return "Z-Score-Plot mit Ein- und Ausstiegspunkten erstellt"

# Plot erstellen
result = create_zscore_plot()
print(result)

Z-Score-Plot mit Ein- und Ausstiegspunkten erstellt
